In [1]:
#load date
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

In [2]:
timeSerials = pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/timeserials.csv',
                      header=0,encoding='gbk' )

In [3]:
def getTimeBatch(start,timegap,splittp):
    x_batch=[]
    for step in range(start,len(splittp)-timegap-1+start):
        x_batch.append(splittp[step:(step+timegap)])
    return x_batch

In [4]:
timegap=30
X = np.array(getTimeBatch(0,timegap,list(timeSerials.buyNum)),dtype=np.float32)
Y = np.array(getTimeBatch(1,timegap,list(timeSerials.buyNum)),dtype=np.float32)
X = X.reshape(X.shape[0],X.shape[1],1)
Y = Y.reshape(Y.shape[0],Y.shape[1],1)
Xtrain = X[:-1]
Ytrain = Y[:-1]
Xtest = np.array([X[-1]])
Ytest = np.array([Y[-1]])

In [5]:
n_steps = timegap
n_inputs = 1
n_neurons = 100
n_outputs = 1
keep_prob = 0.5
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.tanh)
cell = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
rnn_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_outputs = fully_connected(stacked_rnn_outputs, n_outputs,activation_fn=None)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
learning_rate = 0.005
loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()

In [6]:
n_iterations = 100000
 
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):        
        X_batch, y_batch =  Xtrain ,Ytrain 
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 1000 == 0:
            train_mse = loss.eval(feed_dict={X: Xtrain, y: Ytrain})
            test_mse = loss.eval(feed_dict={X: Xtest, y: Ytest})
            print(iteration, "\ttrain MSE:", train_mse, "\ttest MSE:",test_mse)

0 	train MSE: 57142.3 	test MSE: 39803.8
1000 	train MSE: 27746.8 	test MSE: 6950.86
2000 	train MSE: 27349.0 	test MSE: 6392.75
3000 	train MSE: 27254.3 	test MSE: 7566.68
4000 	train MSE: 27054.0 	test MSE: 4670.02
5000 	train MSE: 25751.9 	test MSE: 6476.54
6000 	train MSE: 25067.2 	test MSE: 5342.15
7000 	train MSE: 24465.5 	test MSE: 4832.83
8000 	train MSE: 24309.6 	test MSE: 5323.96
9000 	train MSE: 24289.1 	test MSE: 6106.19
10000 	train MSE: 24182.0 	test MSE: 5095.11
11000 	train MSE: 24108.6 	test MSE: 6369.1
12000 	train MSE: 23356.5 	test MSE: 6154.4
13000 	train MSE: 23383.4 	test MSE: 5841.32
14000 	train MSE: 23088.1 	test MSE: 8825.76
15000 	train MSE: 23093.3 	test MSE: 6371.59
16000 	train MSE: 22689.5 	test MSE: 5026.23
17000 	train MSE: 23002.8 	test MSE: 5272.45
18000 	train MSE: 22387.8 	test MSE: 5854.19
19000 	train MSE: 21428.2 	test MSE: 4778.39
20000 	train MSE: 19669.8 	test MSE: 6012.21
21000 	train MSE: 19951.2 	test MSE: 6363.14
22000 	train MSE: 19009.8

KeyboardInterrupt: 